In [1]:
import click
import datetime
import fnmatch
import logging
import os
import requests
import sys
import telnetlib
import urllib

In [19]:
class Kontrol:
    ipaddr = '192.168.1.1'
    port = 80
    user = 'admin'
    password = ''
    telnet_conn = ''
    urls = dict()
    seqs = dict()
    # throttle (0 is off),rudder (min 1?),elevation (1..127),aileron (1..127),0 + settings (speed mode, inverted flying)
    state = {'throttle':0, 'rudder':1, 'elevation':1, 'aileron':1, 'settings':0}
    var = 250
    
    def __init__(self, ip, port, user, passw):
        self.ipaddr = ip
        self.port = port
        self.user = user
        self.passw = passw
        
    def done(self):
        self.telnet_conn.write(b"exit\n")
        
    def get_comm(self):
        checksum = sum(self.state['throttle'],self.state['rudder'],
                        self.state['elevation'],self.state['aileron'],self.state['settings'])
        abytes = bytes([self.var, self.state['throttle'],self.state['rudder'],
                        self.state['elevation'],self.state['aileron'],self.state['settings'], checksum])
        return abytes
    def seq_store(self, seqname, barray):
        self.seqs[seqname] = barray
        return
    
    def seq_for(self, seqname):
        if seqname in self.seqs:
            return self.urls[script]
        return bytes([0xfa, 0x40, 0x40, 0x40, 0x00, 0x00])
    
    def url_for(self, script):
        if script in self.urls:
            return self.urls[script]
        url = 'http://{addr}:{port}/{script}'.format(
            addr=self.ipaddr, port=self.port, script=script)
        self.urls[script] = url
        return url
        
    def download(self, output, path):
        if output is None:
            output = os.path.basename(path)
            
        res = requests.get(self.url_for('get_record.cgi'),
                           params=dict(path=path, json=1, **auth_params()),
                           stream=True)
        
        res.raise_for_status()
        
        LOG.info('writing %s to %s', path, output)
        with open(output, 'w') as fd:
            for chunk in res.iter_content(chunk_size=8192):
                fd.write(chunk)
        
    def auth_params(self):
        return {'user': self.user, 'pwd': self.password}
    def snapshot(self,output):
        if output is None:
            now = datetime.datetime.now()
            output = 'snapshot-{}.jpg'.format(now.isoformat())
        
        res = requests.get(self.url_for('snapshot.cgi'),
                           params=dict(json=1, **self.auth_params()))  # , **auth_params()
        
        res.raise_for_status()
        
        with open(output, 'wb') as fd:
            fd.write(res.content)
    
    
    


In [20]:
myKontrol = Kontrol('192.168.1.1', 80, 'admin', '')
myKontrol.url_for('get_record.cgi')
myKontrol.snapshot(None)

In [5]:
hex(250)

'0xfa'